In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_path = '/content/gdrive/My Drive/KULIAH SEMESTER 8/KODING PENELITIAN SKRIPSI/'

In [ ]:
import os, sys
sys.path.append(root_path)
sys.path.append(root_path + 'sastrawi_installation')
os.chdir(root_path)

from transformers import BertForSequenceClassification, BertConfig, BertTokenizer

from utils.forward_fn import forward_sequence_classification
from utils.metrics import document_sentiment_metrics_fn
from utils.data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader

In [ ]:
from transformers import AutoTokenizer, AutoModel
# Load Tokenizer and Config
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')
config = BertConfig.from_pretrained('indobenchmark/indobert-base-p1')
config.num_labels = DocumentSentimentDataset.NUM_LABELS

model = BertForSequenceClassification.from_pretrained(root_path + "saved_model/classification_text_indobert", config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_csv(root_path + "data_update.csv")
df_asli = df.copy()
df['content'] = df['content'].astype(str)
df['content'] = df['content'].str.lower()

In [ ]:
import torch
w2i, i2w = DocumentSentimentDataset.LABEL2INDEX, DocumentSentimentDataset.INDEX2LABEL
def sentiment_predict(review):
  subwords = tokenizer.encode(review)
  subwords = torch.LongTensor(subwords).view(1, -1).to(model.device)

  logits = model(subwords)[0]
  label = torch.topk(logits, k=1, dim=-1)[1].squeeze().item()
  return i2w[label]

In [ ]:
df_asli['sentiment'] = df['content'].apply(sentiment_predict)

In [ ]:
df_asli.to_csv(root_path + "dataset_autolabelling_indobert.csv", sep=",", index=False)

In [ ]:
df_asli = pd.read_csv(root_path + "dataset_autolabelling_indobert.csv")

In [ ]:
df_asli[['content', 'sentiment']]

,content,sentiment
0,"Mantap, pelayanan yang sangat istimewa seperti...",positive
1,Sukses terus buat jogjakita,positive
2,Setiap naik jogja kita motor nya tua tua semua...,negative
3,pinaringan sehat lancar rejeki pak,positive
4,mantap,positive
...,...,...
1809,"Coba pembayaran pdam bantul bisa masuk apk ini,",neutral
1810,Mantab,positive
1811,Ok,positive
1812,Akunnya kok dari akun bebasbayar?,neutral


In [ ]:
df_asli['sentiment'].value_counts()

sentiment
positive    879
negative    785
neutral     150
Name: count, dtype: int64